In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# some basic imports
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# After executing the cell above, Drive
# files will be present in "/content/drive/My Drive".
!ls "/content/drive/My Drive/Jordan/Data"

all_data_takaful_and_commercial.dta  Matched_drugs5.csv
choice_drug_not_matched1.csv	     merged_all1415.dta
choice_drug_not_matched2.csv	     new_choice_drugs.csv
classes.dta			     Not_matched_drugs2.csv
combined_data.dta		     Not_matched_drugs3.csv
combined_data_RIDITScores.csv	     outcome.csv
datetest.csv			     percentile_scores2.csv
drug_data.dta			     Plan_Switching_NetworkCoded.csv
Drug_search.xlsx		     s_stats.csv
FinalCodedData.csv		     Stay_high.csv
final_data.csv			     test.csv
FinalSwitchCodedData.csv	     testdata.csv
FinalSwitchCodedData_v1.csv	     transformed_data5.csv
Matched_drugs2.gsheet		     unique_drug_freq.csv
Matched_drugs4.csv


In [4]:
#changing the directory to the data directory
%cd /content/drive/My\ Drive/Jordan/Data

/content/drive/My Drive/Jordan/Data


In [5]:
class DataShell:
  def __init__(self):
    """Initializes the class with some information about the data and the model"""
    self.data_used = 'The data used is the Switch coded data to generate summary statistics'

  def read_data_stata(self,filename):
    self.data_new = pd.read_stata(filename)
    return self.data_stata

  def read_data_csv(self,filename):
    self.data = pd.read_csv(filename)
    return self.data

  def get_initial_data(self):
    """Returns the initial dataframe in its current form"""
    return self.data

  def convert_to_datetime(self,column):
    """Converts a column to a datetime object
       parameters:
              column: The column to be converted to a datetime object
       returns: returns the dataframe with the converted column"""
    self.data[column] = pd.to_datetime(self.data[column],errors='coerce')
    return self.data

  def generate_coverage(self):
    self.data['coverage'] = [1 if network=='VIP' else 2 for network in self.data['network_type']]
    return self.data
  
  def correct_stayhighlow_switchdates(self):
    l1 = ['Addition','Voluntary Deletion','Replacement of Lost Card','Change In Category','Error Correction','Elective Cancellation']
    self.data['Stay_high'] = 0
    self.data['Stay_low'] = 0
    self.data['Pre-Post Switch_down'] = 0
    self.data['Pre-Post Switch_up'] = 0
    i = 0
    k1 = pd.DataFrame(columns=self.data.columns)
    uni_prin_hof = self.data['principal_hof'].unique()
    val_counts = self.data['principal_hof'].value_counts().to_dict()  
    for j,p_hof in enumerate(uni_prin_hof):
      print(j/len(uni_prin_hof)*100)
      k2 = self.data[i:i+val_counts[p_hof]]
      k2.sort_values('endo_date',inplace=True)
      k2 = k2.reset_index(drop=True)
      if k2['VIP_Plan'].tolist().count(1) == len(k2):
        if ('Initial New' or 'Initial Renewal') in k2['endo_type'].unique():
          if any(item in k2['endo_type'].unique() for item in l1)==False:
            k2['Stay_high'] = 1
      if k2['Economy_Plan'].tolist().count(1) == len(k2):
        if ('Initial New' or 'Initial Renewal') in k2['endo_type'].unique():
          if any(item in k2['endo_type'].unique() for item in l1)==False:
            k2['Stay_low'] = 1
      if k2['Switch_up'].tolist().count(1) == len(k2):
        for t in range(len(k2)):
          if ('Change In Category' or 'Error Correction' or 'Elective Cancellation') in k2.loc[t,'endo_type']:
            k2.loc[t:len(k2)-1,'Pre-Post Switch_up'] = 1
            break
      if k2['Switch_down'].tolist().count(1) == len(k2):
        for y in range(len(k2)):
          if ('Voluntary Deletion' or 'Replacement of Lost Card') in k2.loc[y,'endo_type']:
            k2.loc[y:len(k2)-1,'Pre-Post Switch_down'] = 1
            break
      k1 = k1.append(k2)
      i = i + val_counts[p_hof]
    return k1


  

In [6]:
#Creates an object for the class and reads the data from the directory
data_shell = DataShell()
data = data_shell.read_data_csv('FinalCodedData.csv')
#data.head()
#data_shell.DelUnnamedCol()
print("The data has ",data.shape[0]," rows and ",data.shape[1]," atrributes")

AttributeError: ignored

In [ ]:
data = data_shell.data
data.tail()

,payer,master_contract,contract,policynumber,effictive_date,exp_date,endorsment_id,endo_type,endo_date,issue_date,principal_hof,beneficiaryid,beneficiary,card_num,nationality_no,network_type,network,policy_status,empno,dependency,gender,nationality,marital_status,dob,product,category,net_premium,gross_premium,age,currency,mastercontract,branch,reinsurer,effictivedate,expirydate,policyno,maritalstatus,ageband,benef_status,networktype,...,receptiondate,beneficiary_id,authorizationnbr,claimid,invoicenbr,resubmission,providercountry,provideremirate,providertype,providergroup,provider,claimedamount,beneficiartshe_coinsurance,totalcimaiedam,totalpayable,payershare,ben_share_approved,deductionamount,recovery_gartia,recoverytype,payable_non_ip_con,po,orderdate,cheque_nbr,settledate,status,claimcatogory,icdchapter,icd_subchapter,assessment,recoverynote,Switch_down,Switch_up,Addition,VIP_Plan,Economy_Plan,Stay_high,Stay_low,Pre-Post Switch_down,Pre-Post Switch_up
631177,Al Hilal Takaful,Mohamed Abdulrahman Al Bahar (Junior Staff),Mohamed Abdulrahman Al Bahar (Junior Staff) - ...,100-2-NS-X-0835,2014-12-15,2015-12-15,1.0,Initial New,2014-12-15,2014-12-14,Zulfiqar Muhammad,1760821.0,Zulfiqar Muhammad,F7B6-D1BB-378A-035D,NaN,Restricted Network,Hilal Tak.-RN-RES. NET,Policy Expired,NaN,Principal,Male,Pakistan,Married,1-Jan-71,Mohamed 2014 (Jr. Staff) - Dubai,DXB Staff,2257.0,2703.0,43.0,Arab Emirates Dirham,Mohamed Abdulrahman Al Bahar (Junior Staff),Abu Dhabi (H.O),NaN,12/15/2014,12/15/2015,100-2-NS-X-0835,Married,Age 36-45,Active,Restricted Network,...,3/29/2015,F7B6,NaN,19515415,2952,No,United Arab Emirates,Sharjah,Pharmacy,Group Not defined,Bilal Pharmacy - Sharjah,179.68,0.0,179.68,179.64,179.64,0.00,0.04,0.0,,0.0,97288.0,42141.0,145119,6/23/2015,Settled,Paid,Diseases of the respiratory system (ICD9CM 460...,Acute upper respiratory infections (J00-J06)(4...,465.9 Acute Upper Respiratory Infections Of Un...,NaN,0,0,0,0,1,0,1,0,0
631178,Al Hilal Takaful,Mohamed Abdulrahman Al Bahar (Junior Staff),Mohamed Abdulrahman Al Bahar (Junior Staff) - ...,100-2-NS-X-0835,2014-12-15,2015-12-15,1.0,Initial New,2014-12-15,2014-12-14,Zulfiqar Muhammad,1760821.0,Zulfiqar Muhammad,F7B6-D1BB-378A-035D,NaN,Restricted Network,Hilal Tak.-RN-RES. NET,Policy Expired,NaN,Principal,Male,Pakistan,Married,1-Jan-71,Mohamed 2014 (Jr. Staff) - Dubai,DXB Staff,2257.0,2703.0,43.0,Arab Emirates Dirham,Mohamed Abdulrahman Al Bahar (Junior Staff),Abu Dhabi (H.O),NaN,12/15/2014,12/15/2015,100-2-NS-X-0835,Married,Age 36-45,Active,Restricted Network,...,4/16/2015,F7B6,NaN,19523558,3681,No,United Arab Emirates,Sharjah,Pharmacy,Group Not defined,Bilal Pharmacy - Sharjah,134.15,0.0,134.15,134.14,134.14,0.00,0.01,0.0,,0.0,97288.0,42141.0,145119,6/23/2015,Settled,Paid,Diseases of the respiratory system (ICD9CM 460...,Acute upper respiratory infections (J00-J06)(4...,465.9 Acute upper respiratory infections of un...,NaN,0,0,0,0,1,0,1,0,0
631179,Al Hilal Takaful,Mohamed Abdulrahman Al Bahar (Junior Staff),Mohamed Abdulrahman Al Bahar (Junior Staff) - ...,100-2-NS-X-0835,2014-12-15,2015-12-15,1.0,Initial New,2014-12-15,2014-12-14,Zulfiqar Muhammad,1760821.0,Zulfiqar Muhammad,F7B6-D1BB-378A-035D,NaN,Restricted Network,Hilal Tak.-RN-RES. NET,Policy Expired,NaN,Principal,Male,Pakistan,Married,1-Jan-71,Mohamed 2014 (Jr. Staff) - Dubai,DXB Staff,2257.0,2703.0,43.0,Arab Emirates Dirham,Mohamed Abdulrahman Al Bahar (Junior Staff),Abu Dhabi (H.O),NaN,12/15/2014,12/15/2015,100-2-NS-X-0835,Married,Age 36-45,Active,Restricted Network,...,3/29/2015,F7B6,NaN,19502254,3468,No,United Arab Emirates,Sharjah,Pharmacy,Group Not defined,Bilal Pharmacy - Sharjah,164.05,0.0,164.05,164.03,164.03,0.00,0.02,0.0,,0.0,97288.0,42141.0,145119,6/23/2015,Settled,Paid,Diseases of the respiratory system (ICD9CM 460...,Acute upper respiratory infections (J00-J06)(4...,465.9 Acute Upper Respiratory Infections Of Un...,NaN,0,0,0,0,1,0,1,0,0
631180,Al Hilal Takaful,Mohamed Abdulrahman Al Bahar (Junior Staff),Mohamed A

In [ ]:
data = data_shell.convert_to_datetime('endo_date')
data = data_shell.convert_to_datetime('exp_date')
data = data_shell.convert_to_datetime('effictive_date')
data = data_shell.convert_to_datetime('issue_date')
#data_shell.DelUnnamedCol()

In [7]:
data['Pre-Post Switch_up'].value_counts()

0    621930
1      9252
Name: Pre-Post Switch_up, dtype: int64

In [8]:
k = data[data['Switch_up']==1]
k['Pre-Post Switch_up'].value_counts()

1    9252
0    4619
Name: Pre-Post Switch_up, dtype: int64

In [9]:
data['Addition'].value_counts()

0    445968
1    185214
Name: Addition, dtype: int64